In [1]:
using SparseArrays, LinearAlgebra, Random, MAT, PyPlot
Random.seed!(1234)

TaskLocalRNG()

In [2]:
NN = matread("neighbors8810punti.mat")
NN["t"]

17616×3 Matrix{Float64}:
    1.0     2.0     3.0
    1.0     3.0     5.0
    4.0     1.0     5.0
    4.0     5.0     7.0
    6.0     4.0     7.0
    6.0     7.0     9.0
    8.0     6.0     9.0
    8.0     9.0    11.0
   10.0     8.0    11.0
   10.0    11.0    13.0
   12.0    10.0    13.0
   12.0    13.0    15.0
   14.0    12.0    15.0
    ⋮            
 8804.0  8604.0  8805.0
 8606.0  8608.0  8806.0
 8805.0  8606.0  8806.0
 8608.0  8610.0  8807.0
 8806.0  8608.0  8807.0
 8610.0  8612.0  8808.0
 8807.0  8610.0  8808.0
 8612.0  8614.0  8809.0
 8808.0  8612.0  8809.0
 8614.0  8616.0  8810.0
 8809.0  8614.0  8810.0
 8616.0  8561.0  8810.0

In [3]:
#Distance = matread("dist_poli.mat")
#Distance["d_p"]

In [4]:
function sort(NN::Matrix)
    for k = 1:size(NN,1)
        NN[k,:] = Base.sort(NN[k,:],rev = true)
        NN[k,:]
    end
    NN
end

sort (generic function with 1 method)

In [5]:
function size_(N::Matrix)
    M = zeros(Int64, 8810, 7)
    for i = 1:size(N,1)
        M[i,:] = resize!(N[i,:], 7)
        M[i,:]
    end
    M
end

size_ (generic function with 1 method)

In [6]:
function nearest_neighbors(K::Matrix)
    NN = zeros(Int64, 8810, 100000) #Matrix{Int64}[]
    i = 0
    for l = 1:8810
        for j = 1:size(K,1)
            
            if K[j,1]==l
                
                if K[j,2] ∈ NN[l,:]
                    popat!(NN[l,:],i)
                    #i -= 1
                else
                    i += 1
                    NN[l,i] = K[j,2]
                end
                
               
                if K[j,3] ∈ NN[l,:]
                    popat!(NN[l,:],i)
                    #i -= 1
                else
                    i += 1
                    NN[l,i] = K[j,3]
                end
                
            elseif K[j,2]==l
                
                if K[j,1] ∈ NN[l,:]
                    popat!(NN[l,:],i)
                    #i -= 1
                else
                    i += 1
                    NN[l,i] = K[j,1]
                end
                
                if K[j,3] ∈ NN[l,:]
                    popat!(NN[l,:],i)
                    #i -= 1
                else
                    i += 1
                    NN[l,i] = K[j,3]
                end
                
            elseif K[j,3]==l
                
                if K[j,1] ∈ NN[l,:]
                    popat!(NN[l,:],i)
                    #i -= 1
                else
                    i += 1
                    NN[l,i] = K[j,1]
                end
                
                if K[j,2] ∈ NN[l,:]
                    popat!(NN[l,:],i)
                    #i -= 1
                else
                    i += 1
                    NN[l,i] = K[j,2]
                end
                
                
            else
                continue
            end
        end
    end
    
    NN = sort(NN)
    N = size_(NN)
    #N = map(N->round(Int,N), N)
    
end

nearest_neighbors (generic function with 1 method)

In [ ]:
nn = nearest_neighbors(NN["t"])

In [ ]:
nn_ = [nn[i,:] for i in 1:size(nn,1)]
for i = 1:length(nn_)
    j = 1
    while j <= length(nn_[i])
        if nn_[i][j]==0
            popat!(nn_[i], j)
            j -= 1
            nn_[i]
        else 
            j += 1
        end
    end
    nn_[i]
end
nn_

In [ ]:
using CSV, Tables, DataFrames
CSV.write("nn_8810punti.csv", Tables.table(nn_))

In [ ]:
# parameters
nodes = 8810                 #number of nodes (a square of a number for the square lattice)                      
ncol = 1                      #number of protein types (colors)
T = 1e4                         #final time
g = 10                       #aggregation constant (g >1 homologous neighbours reduce the prob )
kI = 1e-5                     #insertion coefficient
const kE = 1e20                      #extraction coefficient 
thrE = 20 #trunc(Int64, (pi/2)*9) #extraction threshold
kD = ones(8810)

t = 0.0                        #initial time
rho = 0.0;                    #constant density

In [ ]:
v = zeros(Int64, nodes)
for j = 1:nodes
    v[j] = count(i->(i != 0), nn[j, :])
end
v;

In [ ]:
function rate(mp,g::Int64=g, kD=kD)           #mp = moves possible
    for i = 1:length(mp)
        return kD[i]*mp[2]/g^mp[1]
    end
end 

In [ ]:
mp = [[h,e] for i=1:length(v) for h=0:v[i]+1 for e=0:v[i]+1 if ((h+e < v[i]+1) & (e>0))];
mp = unique(mp)
mpi = Dict(i => c for (c,i)=enumerate(mp))    #enumerate ritorna il contatore e il valore associato
#println("All possible configurations (nh,ne): \n", mp, "\n")
#println("mpi: \n", mpi)

rates = map(rate, mp);        # function rate(mp)
#print("rates: \n", rates, "\n")

In [ ]:
eta = rand(1:ncol, nodes).*(rand(nodes).<rho);  # state of the system

In [ ]:
#density 
function counting(eta)
    nums = Int64[]
    for i in sort!(unique(eta))
        n = count(x->x==i,eta)
        push!(nums, n)
    end
    return nums
end

nums = counting(eta) #count the number of particles for each species in starting from 0
if length(nums) == 1
    rho_tot = 0.
else
    rho_tot = sum(nums[2:end])/nodes
end
#println("nums " ,nums)
rho_i = zeros(Float64, 1, ncol)

for (ind, col) in enumerate(nums[2:end])
    #println(ind,"  ", col)
    rho_i[ind] = nums[2:end][ind]/nodes
end
rho_tot, rho_i

In [ ]:
eta_nn = copy(nn)
for k = 1:length(eta)
    for i = 1:size(eta_nn,1)
        for j = 1:size(eta_nn,2)
            if eta[k] == 1 && eta_nn[i,j] == k
                eta_nn[i,j] = 1
            elseif eta[k] == 0 && eta_nn[i,j] == k
                eta_nn[i,j] = 0
            end
            eta_nn[i,j]
        end
    end
end
eta_nn;

In [ ]:
eta_ = [eta[nn_[i]] for i = 1:length(nn_)]
ne = (v - [sum(eta_[i]).>0 for i=1:length(eta_)]).*(eta.>0)  #array of empty neighbors for each node
eta1 = repeat(reshape(eta, nodes ,1), 1, 7)
nh = sum((eta1 .== eta_nn) .& (eta1.>0) ,dims = 2);  #array of homologous neighbors for each node
#println("number of empty neighbors ", ne, "\n")
#println("eta1 is a multi-column-version of the eta matrix: \n", eta1, "\n")
#println("number of homologhe neighbors ", nh)

m = hcat(nh,ne);   #Matrix nodesx2

In [ ]:
sd = Vector{Int64}[]               #controllo se per il nodo j ci sono mosse possibili,
isd = -ones(Int64, nodes)          #se sì, aggiungo j alla riga i-esima di sd
@inbounds for i in 1:length(mp)
    push!(sd, Int64[])
    for j in 1:nodes
        if all(m[j,:] .== mp[i])    #Determine whether all elements of m[j,:] are equal to mp[i], returning false as soon as the first item in itr for which p returns false is encountered
            append!(sd[i],j)    #add element j to the end of sd
            isd[j] = length(sd[i])
        end
    end
end

#nodes where there is not particle prensent
pel = -ones(Int64, nodes)    #pel = position of the empty nodes in the empty list
el = Int64[]
for e in 1:nodes           #This loop can be parallelized                
    if eta[e] == 0                 #controllo se i nodi sono vuoti, se sì li inserisco
        push!(el, e)               #nel vettore el
        pel[e] = length(el)
    end
end
    
nₚ = 0.   # number of monomers
Nd = 0    # number of domains
nd = 0.   # number of molecules inside the clusters (density)
ns = 0.   # number of molecules inside the surface of a clusters (density)
Nin = 0   #number of insertions
Next = 0  # number of extracted domains
Ndis = 0  # number of evaporations (domain that evaporates)
next = 0. # number  of molecules inside an extracted cluster (is approximately equal to Next*thrE)
τₚ = 0.    #productive domain timelife
τᵤ = 0.    #unproductive domain timelife
pos = 0
lcc = Vector{Int64}[]        #= list of connected components without distinction of the col.
                Each sub-array contains the position of the particles belonging to that cluster into i =#
cc = -ones(Int64, nodes)     # it indicates the cluster (sub-array) where it is the particle
icc = -ones(Int64, nodes)    # it gives us the position of the particle inside the sub-arrays 

lccS = Vector{Int64}[] #perimeters of the connected components
iccS = -ones(Int64, nodes) # it gives us the position of the particle inside the sub-arrays
tin = Float64[]
for i in 1:nodes
    if (eta[i] != 0) & (cc[i] == -1)
        #println(i)
        pos += 1
        cc[i] = pos
        icc[i] = 1
        push!(lcc, [i])
        l = length(lcc)
        if sum(eta_[i] .== eta[i]) != v[i]  #particle on the surface
            iccS[i] = 1
            push!(lccS, [i])
        else
            push!(lccS, [])
        end
        @inbounds for j in lcc[l]      #si prende un nodo del cluster l
            @inbounds for k in nn_[j]  #per il nodo j si prende un nodo vicino
                if (eta[k] == eta[i]) & (cc[k] == -1)
                    #print(nn[j,:])
                    cc[k] = pos
                    push!(lcc[l], k)
                    icc[k] = length(lcc[l]) 
                    
                    if sum(eta_[k] .== eta[i]) != v[i]   #particle on the surface
                    push!(lccS[l], k)
                    iccS[k] = length(lccS[l])
                    end 
                end  
            end
        end
        if length(lcc[l]) > 1
            push!(tin, t)
            Nd += 1
            nd += length(lcc[l])/nodes  #num of particles inside a cluster
            ns += length(lccS[l])/nodes #number of particles in the surface of a cluster
        else
            nₚ += 1/nodes
            push!(tin, -1.)
        end
    end    
end

#clusters over size
over_thrE = findall(x->length(x)>=thrE,lcc);

In [ ]:
function choose_move(qs)
    rd = rand(Float64)
    move = 1
    #println(rd)
    while move <= length(qs)
        #println(move)
        if rd < qs[move]
            if move < length(qs) - 1
                return move, "diffusion"
            elseif move < length(qs)
                return move, "insertion"
            else 
                return move, "extraction"
            end    
        else
            move += 1
        end    
    end
end

In [ ]:
function diffusion(move, sd, eta, nn_=nn_::Array{Array{Int64,1},1}) 
    pc = rand(sd[move])                            # 2) random choice of the particle to move
    rc = rand(nn_[pc][findall(x->x==0, eta_[pc])]) # 3) select an empty neighbors of pc at random
    #update the system
    eta[rc] = eta[pc]       #move particle pc in site rc
    eta[pc] = 0
    
    return pc, rc, eta
end

In [ ]:
function insertion(el, eta, ncol::Int64=ncol)
    nc = rand(el)          # 2) choose a node at random (nc = chosen node)
    col = rand(1:ncol)      # 3) choose the protein type to insert
    #update the system
    eta[nc] = col
    
    return nc, col, eta
end

In [ ]:
function extraction(over_thrE::Array{Int64,1}, eta::Array{Int64,1}, lcc=lcc::Array{Array{Int64, 1}, 1})
    ceot = rand(over_thrE)         #ceot = select a cluster from over_thrE (position of the cluster in sd)
    col = eta[lcc[ceot]][1]        #color of the cluster that has to be extracted 
    #update the system
    eta[lcc[ceot]] .= 0  #extraction
    
    return ceot, col, eta
end

In [ ]:
function create_m_loc(lnm, eta, nn_=nn_::Array{Array{Int64, 1}, 1}, v=v::Array{Int64,1})
    #common part
    ne_loc = (v[lnm] - [sum(eta_[lnm][i].>0) for i = 1:length(eta_[lnm]), dims=2]).*(eta[lnm].>0) #array of empty neighbours for each node
    eta1 = repeat(reshape(eta[lnm], length(lnm), 1), 1, 7)
    nh_loc = sum((eta1.==eta_nn[lnm,:]) .& (eta1.>0), dims=2);  #array of homologous neighbours for each node
    m_loc = hcat(nh_loc,ne_loc)
    #println("m_loc: \n", m_loc)
    return m_loc
end

In [ ]:
function update_local_sd_isd(lnm, m, m_loc, sd, isd, mp=mp::Array{Array{Int64, 1}, 1}, mpi=mpi::Dict{Vector{Int64}, Int64})
    @inbounds for i in lnm
        if m[i,:] in mp                 #this use m and not m_loc, for retrieve the particles
            ind_sd = mpi[m[i,:]]
            pos_in_sd = isd[i]   #position in sd
            sd[ind_sd][pos_in_sd] = sd[ind_sd][end]
            isd[sd[ind_sd][end]] = pos_in_sd
            isd[i] = -1
            pop!(sd[ind_sd])
        end
    end
    @inbounds for (i,x) in enumerate(lnm)
        if m_loc[i,:] in mp
            ind_sd = mpi[m_loc[i,:]]
            append!(sd[ind_sd], x) 
            isd[x] = length(sd[ind_sd])  #c'era i
        end
    end
    return sd, isd
end 

In [ ]:
function update_local_el_pel_insertion(nc, el, pel)
    el[pel[nc]] = el[end]
    l = pop!(el)
    pel[l] = pel[nc]
    pel[nc] = -1
    return el, pel
end

In [ ]:
function update_local_el_pel_diffusion(pc, rc, el, pel)
    el[findall(x->x==rc, el)] .= pc
    pel[pc] = pel[rc]
    pel[rc] = -1
    return el, pel
end

In [ ]:
function update_local_el_pel_extraction(ceot, el, pel, lcc)
    @inbounds for i in lcc[ceot]
        push!(el, i)
        pel[i] = length(el)
    end
    return el, pel
end

In [ ]:
function update_local_lcc_cc_icc_insertion(lcc, cc, icc, over_thrE, lccS, iccS, eta, nc, nₚ, nd, Nd, ns, tin, t,
                                            nn_=nn_::Array{Array{Int64, 1}, 1}, thrE::Int64=thrE, nodes::Int64=nodes)
    if eta[nc] ∉ eta_[nc] #1) isolated particle (monomer) 
        #println("homologous neighbors = 0")
        push!(lcc, [nc])
        push!(tin, -1)
        cc[nc] = length(lcc)
        icc[nc] = 1
        push!(lccS, [nc])
        iccS[nc] = 1
        nₚ += 1/nodes 
   
    elseif sum(eta[nc] .== eta_[nc]) == 1     # 2) homologous particle in the neighbourhood = 1
        #println("homologous neighbors = 1")
        ## I want to find the neighbour cluster
        plcc = cc[nn_[nc]][eta_[nc] .== eta[nc]][1]  #position in lcc
        ccn = copy(lcc[plcc])  #ccn = connected component neighbours
        #print("ccn: ", ccn)
        append!(lcc[plcc], nc)
        icc[nc] = length(lcc[plcc])
        cc[nc] = plcc
        if length(ccn) == 1 #join of two monomers
            #println("cluster: ", lcc[plcc])
            tin[plcc] = t    #the time where a new domain is formed
        end       
        
        nn1 = [nn[nc,i] for i = 1:length(nn[nc,:]) if nn[nc,i]!=0]
        an = nn1[eta_[nc] .== eta[nc]][1]
        #an = nn_[nc,eta_[nc] .== eta[nc]][1]                              #  
        #println("adjecient node to nc (an): ", an)                       #  
        if sum(eta_[an] .== eta[nc]) == v[nc] #(???)  #particle in the interior  #
            lccS[cc[an]][iccS[an]] = nc                                   #   
            iccS[nc] = iccS[an]                                           #
            iccS[an] = -1                                                 #
        else                                                              #    
            append!(lccS[plcc], nc)                                       #  perimeter 
            iccS[nc] = length(lccS[plcc])                                 # 
        end                                                               #  
       
        #update over_thrE
        if (length(lcc[plcc]) >= thrE) & (plcc ∉ over_thrE)
            push!(over_thrE, plcc)
        end
        #uptade nₚ, Nd and nd
        #print("ccn: ", ccn)
        #print("lcc[plcc]:",lcc[plcc])
        if length(ccn) == 1
            nₚ -= 1/nodes
            Nd += 1
            nd += length(lcc[plcc])/nodes
            ns += length(lccS[plcc])/nodes  #forse è più efficiente con 2/N
        else
            nd += 1/nodes
            if sum(eta_[an]) != v[nc] #(???)  
                ns += 1/nodes                
            end
        end
        #print("np: ", nₚ, "   nd: ", nd, "  Nd: ", Nd)
        
        
        
    else
        #3) the new particle can marge two or more clusters (nh[nc] >= 2)
        #println("homologous neighbors => 2")
        cn = cc[nn_[nc][eta_[nc] .== eta[nc]]] #which clusters belonging to the homologous neighbors
        #println("lcc[cn]: ", lcc[cn])
        clusters = unique(cn)
        
        len_cl = map(x -> length(x), lcc[clusters]) 
        num_dom = sum(len_cl .> 1)
        #print("num_dom", num_dom)
        
         #different clusters in the neighborhood 
        clusters = sort!(clusters, rev=true)
        #println("clusters: ", clusters)
        
        #println(clusters)
        merged_cl = [nc]  #the new bigger cluster has to contain the particle nc
        merged_clS = []
        if sum(eta_[nc] .== eta[nc]) != v[nc] 
            push!(merged_clS, nc)
        end
        times_rm = Float64[]
        @inbounds for cl in clusters
            if length(lcc[cl]) >= thrE
                over_thrE[cl] = over_thrE[end]
                pop!(over_thrE)
            end
            if length(lcc[cl]) > 1      #update nd, nₚ 
                nd -= length(lcc[cl])/nodes
                ns -= length(lccS[cl])/nodes
            else
                nₚ -= 1/nodes
            end    
            
            merged_cl = append!(merged_cl, lcc[cl]) # create a bigger cluster
            merged_clS = append!(merged_clS, lccS[cl])
            #print("merged_cl: ",merged_cl)
    
            if lcc[cl] != lcc[end]
                cc[lcc[end]] .= cc[lcc[cl]][1]
                cc[lcc[cl]][1] = -1
                lcc[cl] = lcc[end]
                tin[cl] = tin[end]
                pop!(lcc)
                t_rm = pop!(tin)                  #remove formation time
                
                
                lccS[cl] = lccS[end]
                pop!(lccS)
            else
                cc[lcc[end]][1] = -1
                pop!(lcc)                #remove the old cluster 
                pop!(lccS)
                t_rm = pop!(tin)                #remove formation time
            end
            if t_rm != -1. 
                append!(times_rm, t_rm)
            end
            
        end
        #println("removed times ", times_rm)
        if times_rm == []
            t_form = t
        else
            t_form = minimum(times_rm)
        end
        
        #find particle in the cluster interior
        #merged_clS = copy(merged_cl)
        eta2 = [eta[nn_[nn_[nc]][i]] for i = 1:length(nn_[nn_[nc]])]
        p_in = nn_[nc] .* ([sum(eta2[i]) for i = 1:length(eta2)] .== v[nc])
        p_in = p_in[p_in .> 0]
        #println("internal particels: ", p_in)
        
        iccS[p_in] .= -1
        for i in p_in
            filter!(x->x≠i, merged_clS)
        end
        #println("lccS: ", lccS)
        
        push!(lccS, merged_clS)
        push!(lcc, merged_cl)    #add the new cluster
        push!(tin, t_form)            # formation time
        nd += length(merged_cl)/nodes
        ns += length(merged_clS)/nodes
        if num_dom == 0 
            Nd += 1
        elseif num_dom > 1
            Nd -= (num_dom -1) 
        end
        if length(merged_cl) >= thrE    
            push!(over_thrE, length(lcc))
        end
        
        cc[lcc[end]] .= length(lcc)
        #println(merged_cl)

        for (ind, p) in enumerate(merged_cl)
            icc[p] = ind
        end
        for (ind, p) in enumerate(merged_clS)   #
            iccS[p] = ind                       # perimeter
        end                                     #
    end
    return lcc, cc ,icc, over_thrE, lccS, iccS, nₚ, nd, Nd, ns, tin
end

In [ ]:
function update_local_lcc_cc_icc_diffusion(lnm, lcc, cc, icc, over_thrE, lccS, iccS, eta, pc, rc, nₚ, nd, Nd,
                                            ns, tin, t, τᵤ, Ndis, nn_=nn_::Array{Array{Int64, 1}, 1}, thrE::Int64=thrE)
    #remove changed clusters
    lpcr =Int64[]           #list with the positions of the clusters to remove
    @inbounds for i in lnm
        if ((cc[i] != -1) & (eta[i] == eta[rc])) | (i == pc)  #cc[i] != -1 is equivalent to (eta[i] != 0) 
            append!(lpcr, cc[i])
        end
    end 
    lpcr = sort!(unique(lpcr), rev=true)
    #println("lpcr: ", lpcr)
    times_rm = Float64[]
    lcc_rm = []
    cl_rm = []
    @inbounds for i in lpcr
        if length(lcc[i]) >= thrE
            p = findall(x->x==i, over_thrE)[1] #update over_thrE
            over_thrE[p] = over_thrE[end]
            pop!(over_thrE)
        end
        if length(lcc[i]) > 1    #update nₚ, nd, Nd
            nd -= length(lcc[i])/nodes
            ns -= length(lccS[i])/nodes
            Nd -= 1
        else
            nₚ -= length(lcc[i])/nodes
        end
        if lcc[i] != lcc[end]
            cl_rm = lcc[i]  #cluster that will be removed
            t_rm = tin[i]
            tp_cc = cc[lcc[i]][1]
            cc[lcc[i]] .= -1
            cc[lcc[end]] .= tp_cc
            lcc[i] = lcc[end]
            lccS[i] = lccS[end]
            tin[i] = tin[end]
            pop!(lcc)
            pop!(lccS)
            pop!(tin)   #remove formation time
        else
            cc[lcc[end]] .= -1
            cl_rm = pop!(lcc)  #rm_cl = cluster removed
            pop!(lccS)
            t_rm = pop!(tin)
        end 
        if (t_rm != -1.) #&& (pc ∉ cl_rm)
            append!(times_rm, t_rm)
            append!(lcc_rm, [cl_rm])
        end
    end
    #println("removed times ", times_rm)
    #println("removed clusters ", lcc_rm)
    
    icc[pc] = -1
    iccS[nn_[rc]] .= -1 
    #println("nₚ: ", nₚ)
    #println("nd: ", nd)
    #println("Nd: ", Nd)
    
    
    #add new clasters
    tlᵤ = Float64[]
    colors = Int64[]
    starting_points = [k for k in nn_[pc] if eta[rc] == eta[k]]
    @inbounds for s in starting_points
        if cc[s] ∉ colors
            c = length(lcc) + 1
            cc[s] = c
            icc[s] = 1
            push!(colors, c)
            push!(lcc, [s])          
            if sum(eta_[s] .== eta[s]) != v[s]  #particle on the surface            
                push!(lccS, [s])
                iccS[s] = 1
            end
            @inbounds for j in lcc[c]
                @inbounds for k in nn_[j]
                    if (eta[k] == eta[j]) & (cc[k] ∉ colors)
                        push!(lcc[c], k)
                        if sum(eta_[k] .== eta[k]) != v[k]
                            push!(lccS[c], k)
                            iccS[k] =length(lccS[c])
                        end
                        cc[k] = c
                        icc[k] =length(lcc[c]) 
                    end
                end               
            end
            if (length(lcc[c]) >= thrE) & (length(lcc) ∉ over_thrE)         
                push!(over_thrE, length(lcc))
            end 
            if length(lcc[c]) > 1
                if sum([(cl ∩ lcc[c]) ⊆ lcc[c] for cl in lcc_rm if (length(cl) > 1 && !isempty(cl ∩ lcc[c]))] .> 0) > 0
                                
                   pos = findall([(cl ∩ lcc[c]) ⊆ lcc[c] && (length(cl) > 1 && !isempty(cl ∩ lcc[c])) for cl in lcc_rm ])
                    #println("pos ", pos)
                    #println("cluster: ", lcc[c], "     time: ", minimum(times_rm[pos]), "     rm clusters: ", lcc_rm[pos])
                    push!(tin, minimum(times_rm[pos])) #add domain formation                 
                else
                    push!(tin, t)
                    #println(lcc[c]) 
                    #println("time: ", t)
                end
                Nd += 1
                nd += length(lcc[c])/nodes  #num of particles inside domains
                ns += length(lccS[c])/nodes            
                #println("Nd: ", Nd)
            else
                nₚ += 1/nodes
                push!(tin, -1.)             
                # unproductive domains timelife 
                #println("lcc[c]: ", lcc[c])
                if length(times_rm[findall(lcc[c] .∈ lcc_rm)]) > 0 
                                
                    τᵤ = t - times_rm[findall(lcc[c] .∈ lcc_rm)][1]
                    #println("timelife of unproductive domain: ", τᵤ)
                    push!(tlᵤ, τᵤ)
                      
                end     
            end    
        end
    end 
    if length(tlᵤ) > 0
        τᵤ = unique!(tlᵤ)[1]                                        
        Ndis += 1
    else
        τᵤ = 0.
    end                                                
                
    return lcc, cc, icc, over_thrE, lccS, iccS, nₚ, nd, Nd, ns, tin, τᵤ, Ndis            
end

In [ ]:
function update_local_lcc_cc_icc_extraction(ceot, lcc, cc, icc, over_thrE, lccS, iccS,
                                             nₚ, nd, Nd, ns, Next, next, tin, t, τₚ)
    
    nd -= length(lcc[ceot])/nodes #update nd
    ns -= length(lccS[ceot])/nodes
    Nd -= 1
    Next += 1 
    next += length(lcc[ceot])/nodes
    
    if lcc[ceot] != lcc[end]
        cc[lcc[ceot]] .= -1
        icc[lcc[ceot]] .= -1
        lcc[ceot] = lcc[end]
        t_rm = tin[ceot]
        tin[ceot] = tin[end]
        cc[lcc[end]] .= ceot
        pop!(lcc)
        pop!(tin)
        
        iccS[lccS[ceot]] .= -1        #
        lccS[ceot] = lccS[end]        # perimeter
        pop!(lccS)                    #  
    else
        last = pop!(lcc)
        t_rm = pop!(tin)                  #remove formation time
        cc[last] .= -1
        icc[last] .= -1 
         
        lastS = pop!(lccS)            # perimeter
        iccS[lastS] .= -1              #
    end  
    τₚ= t - t_rm    #productive residence time
    
    pceot = findall(x->x==ceot, over_thrE)#position of ceot inside over_thrE
    #println("pceot: ",pceot)
    over_thrE[pceot] .= over_thrE[end] 
    pop!(over_thrE)
    

    #println("ok2")
    
    return lcc, cc, icc, over_thrE, lccS, iccS, nₚ, nd, Nd, ns, Next, next, tin, τₚ
end

In [ ]:
function statistics_1species(eta,t, rho_tot, nₚ, nd, Nd, ns, Next, next, cont, dt, sum_dt, sum_rho_dt, 
                             sum_rho2_dt, sum_rho3_dt, sum_rho4_dt, sum_nₚ_dt, sum_nd_dt, sum_Nd_dt, sum_ns_dt, 
                             sum_τₚ, τₚ, Ndis, sum_τᵤ, τᵤ, Nin, T=T, nodes=nodes, 
                             thrE=thrE, g=g, ncol=ncol, kI=kI; points = 300000)
    
    sum_τₚ += τₚ
    sum_τᵤ += τᵤ
    
    sum_dt += dt
    sum_rho_dt += dt*rho_tot          
    sum_rho2_dt += dt*rho_tot^2
    sum_rho3_dt += dt*rho_tot^3
    sum_rho4_dt += dt*rho_tot^4
    
    sum_nₚ_dt += dt*nₚ 
    sum_nd_dt += dt*nd
    sum_Nd_dt += dt*Nd
    sum_ns_dt += dt*ns
    cont += 1
    
    if cont == points
        avg_τₚ = sum_τₚ/Next              #avg productive lifetime 
        avg_τᵤ = sum_τᵤ/Ndis              #avg unproductive lifetime 
        avg_rho = sum_rho_dt/sum_dt
        avg_rho2 = sum_rho2_dt/sum_dt
        avg_rho3 = sum_rho3_dt/sum_dt
        avg_rho4 = sum_rho4_dt/sum_dt
        
        avg_nₚ = sum_nₚ_dt/sum_dt
        avg_nd = sum_nd_dt/sum_dt
        avg_Nd = sum_Nd_dt/sum_dt
        avg_ns = sum_ns_dt/sum_dt
        avg_next = next/Next
        avg_rate_Next = Next/sum_dt    #extraction rate
        avg_rate_Nin = Nin/sum_dt      #insertion rate
        avg_rate_Ndis = Ndis/sum_dt     #dissolution rate
        #println("STATISTICS")
        #println("avg_rho2 ",avg_rho2, "\n")
        
        var = avg_rho2 - avg_rho^2     #E[x^2] - E^2[x]
        skew = avg_rho3 - 3*avg_rho2*avg_rho + 2*avg_rho^3
        kurt = avg_rho4 - 4*avg_rho3*avg_rho + 6*avg_rho2*avg_rho^2 - 3*avg_rho^4
        #println("weighted avg density: ", avg_rho , "\n")
        #println("first: ", avg_rho2, "second",avg_rho^2,"\n")
        #println("variance: ", var, "\n")
        #println("skewness: ", skew, "\n")
        #println("kurtosis: ", kurt, "\n")
        #write_on_file(eta, t, avg_rho, var, skew, kurt, avg_nₚ, avg_nd, avg_Nd, 
        #              avg_ns, avg_rate_Next, avg_next, avg_τₚ, avg_τᵤ, avg_rate_Ndis, avg_rate_Nin)
        cont = 0
        sum_dt = 0.
        sum_rho_dt = 0.       
        sum_rho2_dt = 0.     
        sum_rho3_dt = 0.      
        sum_rho4_dt = 0.
        sum_nₚ_dt = 0.
        sum_nd_dt = 0.
        sum_Nd_dt = 0.
        sum_ns_dt = 0.
        Next = 0
        next = 0.
        Ndis = 0
        Nin = 0
        sum_τₚ = 0.
        sum_τᵤ = 0.
    end
    return cont, sum_dt, sum_rho_dt, sum_rho2_dt, sum_rho3_dt, sum_rho4_dt, 
           sum_nₚ_dt, sum_nd_dt, sum_Nd_dt, sum_ns_dt,
           Next, next, sum_τₚ, τₚ, Ndis, sum_τᵤ, τᵤ, Nin
end

In [ ]:
#=function statistics_Nspecies(eta, t,  rho_tot, nₚ, nd, Nd, ns, Next, next, rho_i, cont, dt, sum_dt, sum_rho_dt, sum_rho2_dt, sum_rho3_dt,
                             sum_rho4_dt, sum_rho_i1_dt, sum_rho_i2_dt, sum_rho_i12_dt,sum_nₚ_dt, sum_nd_dt, 
                             sum_Nd_dt, sum_ns_dt, sum_τₚ, τₚ, Ndis, 
                             sum_τᵤ, τᵤ, Nin, T=T, nodes=nodes, 
                             thrE=thrE, g=g, ncol=ncol, kI=kI; points = 300000)
    
    sum_τₚ += τₚ 
    sum_τᵤ += τᵤ
    
    sum_dt += dt
    sum_rho_dt += dt*rho_tot          
    sum_rho2_dt += dt*rho_tot^2
    sum_rho3_dt += dt*rho_tot^3
    sum_rho4_dt += dt*rho_tot^4
    sum_rho_i1_dt += rho_i[1]*dt
    sum_rho_i2_dt += rho_i[2]*dt
    sum_rho_i12_dt += rho_i[1]*rho_i[2]*dt
    
    sum_nₚ_dt += dt*nₚ 
    sum_nd_dt += dt*nd
    sum_Nd_dt += dt*Nd
    sum_ns_dt += dt*ns
    cont += 1
    
    if cont == points
        avg_τₚ = sum_τₚ/Next              #avg productive lifetime 
        avg_τᵤ = sum_τᵤ/Ndis              #avg unproductive lifetime 
        avg_rho = sum_rho_dt/sum_dt
        avg_rho2 = sum_rho2_dt/sum_dt
        avg_rho3 = sum_rho3_dt/sum_dt
        avg_rho4 = sum_rho4_dt/sum_dt
        avg_rho_i1 = sum_rho_i1_dt/sum_dt
        avg_rho_i2 = sum_rho_i2_dt/sum_dt
        avg_rho_i12 = sum_rho_i12_dt/sum_dt
        
        avg_nₚ = sum_nₚ_dt/sum_dt
        avg_nd = sum_nd_dt/sum_dt
        avg_Nd = sum_Nd_dt/sum_dt
        avg_ns = sum_ns_dt/sum_dt
        avg_next = next/Next
        avg_rate_Next = Next/sum_dt    #extraction rate
        avg_rate_Nin = Nin/sum_dt      #insertion rate
        avg_rate_Ndis = Ndis/sum_dt     #dissolution rate
        #println("STATISTICS")
        #println("avg_rho2 ",avg_rho2, "\n")
        #println("avg_rho_i1 ", avg_rho_i1, "  avg_rho_i2: ", avg_rho_i2,"\n")
        #println("avg_rho_i12: ", avg_rho_i12, "\n")
        
        var = avg_rho2 - avg_rho^2     #E[x^2] - E^2[x]
        skew = avg_rho3 - 3*avg_rho2*avg_rho + 2*avg_rho^3
        kurt = avg_rho4 - 4*avg_rho3*avg_rho + 6*avg_rho2*avg_rho^2 - 3*avg_rho^4
        cov = avg_rho_i12 - avg_rho_i1*avg_rho_i2
        
        #println("weighted avg density: ", avg_rho , "\n")
        #println("first: ", avg_rho2, "second",avg_rho^2,"\n")
        #println("variance: ", var, "\n")
        #println("skewness: ", skew, "\n")
        #println("kurtosis: ", kurt, "\n")
        #print("covariance between rho_1 and rho_2: ", cov, "\n")
        write_on_file(eta, t, avg_rho, var, skew, kurt, avg_nₚ, avg_nd, avg_Nd, avg_ns,
                      avg_rate_Next, avg_next, avg_τₚ, avg_τᵤ, avg_rate_Ndis, avg_rate_Nin, avg_rho_i1, avg_rho_i2, cov)
    end
        
        cont = 0
        sum_dt = 0.
        sum_rho_dt = 0.       
        sum_rho2_dt = 0.     
        sum_rho3_dt = 0.      
        sum_rho4_dt = 0.
        sum_rho_i1_dt = 0.
        sum_rho_i2_dt = 0.
        sum_rho_i12_dt = 0. 
        sum_nₚ_dt = 0.
        sum_nd_dt = 0.
        sum_Nd_dt = 0.
        sum_ns_dt = 0.
        Next = 0
        next = 0.
        Ndis = 0
        Nin = 0
        sum_τₚ = 0.
        sum_τᵤ = 0.
        
    return cont, sum_dt, sum_rho_dt, sum_rho2_dt,sum_rho3_dt, sum_rho4_dt, 
           sum_rho_i1_dt, sum_rho_i2_dt, sum_rho_i12_dt, sum_nₚ_dt, sum_nd_dt, 
           sum_Nd_dt, sum_ns_dt, Next, next, sum_τₚ, τₚ, 
           Ndis, sum_τᵤ, τᵤ, Nin 
end =#

In [ ]:
#= function write_on_file(eta, t, avg_rho, var, skew, kurt, avg_nₚ, avg_nd, avg_Nd, avg_ns, avg_rate_Next, avg_next, 
                       avg_τₚ, avg_τᵤ, avg_rate_Ndis, avg_rate_Nin, avg_rho_i1 = NaN, avg_rho_i2= NaN, cov= NaN, T::Float64=T, 
                       N::Int64=nodes, thrE::Int64=thrE, g::Int64=g, ncol::Int64=ncol, 
                       kI::Float64=kI)
    #data series
    mol_sorting = open("mol_sorting(ncol_$(ncol)_g_$(g)_kI_$(kI)_thrE_$(thrE))", "a")
    
    data = "$t" *"    " * "$avg_rho" * "    " * "$var" * "    " * "$skew" * 
            "    " * "$kurt" * "    " * "$avg_rho_i1" * "    " * "$avg_rho_i2" * "    " * "$cov" *
            "    " * "$avg_nₚ" * "    " * "$avg_nd" * "    " * "$avg_Nd"  * "    "* "$avg_ns" * "    " * 
            "$avg_rate_Next" * "    " * "$avg_next"* "    " * "$avg_τₚ" * "    " * "$avg_rate_Ndis" * 
            "    " * "$avg_rate_Nin" * "    " * "$avg_τᵤ\n"
    write(mol_sorting, data)
    
    close(mol_sorting)
    #imshow
    system = open("imshow(ncol_$(ncol)_g_$(g)_kI_$(kI)_thrE_$(thrE))", "w")
    #system = "imshow(ncol_$(ncol)_g_$(g)_kI_$(kI)_thrE_$(thrE)type_lattice_$(type_lattice)).jld"
    #save(system, eta, "w")
    write(system, "$eta \n" )
    close(system)
end =#

In [ ]:
#statistics
cont = 0
sum_dt = 0.           
sum_τₚ = 0.           #productive domain lifetimes
sum_τᵤ = 0.           #unproductive domain lifetimes

sum_rho_dt = 0.       #rho*t
sum_rho2_dt = 0.      #rho**2*t
sum_rho3_dt = 0.      #rho**3*t
sum_rho4_dt = 0.      #rho**4*t
if ncol > 1
    sum_rho_i1_dt = 0.
    sum_rho_i2_dt = 0.
    sum_rho_i12_dt = 0.              #rho_i1*rho_i2*t
end
sum_nₚ_dt = 0.
sum_nd_dt = 0.
sum_Nd_dt = 0.
sum_ns_dt = 0.

In [ ]:
#main  
densit = []
dt = 0
n = zeros(Int64, nodes)
#file = open("cluster_with_$(nodes)_nodes.txt", "w")
while (t < T)
    #probabilities
    qD = rates.*map(length, sd)
    #qD1 = reduce(vcat, qD)
    qI = kI*length(el)
    qE = kE*length(over_thrE)
    q =  vcat(qD,qI,qE)
    qs = cumsum(q)
    qs /= qs[end]  #normalization
    
    
    #println()
    #println("Time: ", t)
    #println("-----------------------------------------------------------------------------")
    #println("Not normalized probability of extraction: \n", qE, "\n")
    #println("Not normalized probability of diffusion: \n", qD, "\n")
    #println("Not normalized probability of insertion: \n", qI, "\n")
    #println("All not normalized prob: diffusion, insertion, extraction \n", q ,"\n")
    #println("Normalized probabilities: \n", qs, "\n")
    
    # 1) it chooses the type of move
    move, type_of_move = choose_move(qs)  
    #println("type of move: ", type_of_move)
    #dt = -log(rand(Float64))/sum(q)
    #t = t + dt
    #=--------------------------------------DIFFUSION-------------------------------------------------=#
    if type_of_move == "diffusion"
        
        pc, rc, eta = diffusion(move, sd, eta)
        #println("The particle goes from ", pc, " to ", rc, "\n")
        
        eta_ = [eta[nn_[i]] for i = 1:length(nn_)]
        
        rho_tot = rho_tot                          #diffusion does not change the density of the system
        #rho_i = rho_i
         
        lnm = unique(vcat(nn_[pc],nn_[rc]))                                # array of sites to update
        #println("lnm diff")
        m_loc = create_m_loc(lnm, eta)                #the same for insertion, diffusion and extraction
        #println("m_loc diff \n", m_loc,"\n")
        sd, isd = update_local_sd_isd(lnm, m, m_loc, sd, isd) # the same for insertion, diffusion and extraction
        el, pel = update_local_el_pel_diffusion(pc, rc, el, pel)
        lcc, cc, icc, over_thrE, lccS, iccS, nₚ, nd, Nd, ns,tin, τᵤ, Ndis = update_local_lcc_cc_icc_diffusion(lnm, lcc, cc, 
                                             icc, over_thrE, lccS, iccS, eta, pc, rc, nₚ, nd, Nd, ns, tin, t, τᵤ, Ndis)
        m[lnm,:] = m_loc
    #=--------------------------------------INSERTION-------------------------------------------------=#
    elseif  type_of_move == "insertion"
        nc, col, eta = insertion(el, eta)  
        
        eta_ = [eta[nn_[i]] for i = 1:length(nn_)]
        
        Nin += 1
        rho_tot +=  1. /nodes
        #rho_i[col] += 1. /nodes
         
        lnm = vcat(nc,nn_[nc]) # array of sites to update
        #println("lnm ins")
        m_loc = create_m_loc(lnm, eta) #the same for insertion, diffusion and extraction
        #println("m_loc ins \n", m_loc,"\n")
        sd, isd =update_local_sd_isd(lnm, m, m_loc, sd, isd) # the same for insertion, diffusion and extraction
        el, pel = update_local_el_pel_insertion(nc, el, pel)
        lcc, cc, icc, over_thrE, lccS, iccS, nₚ, nd, Nd , ns, tin = update_local_lcc_cc_icc_insertion(lcc, cc, icc, 
                                                        over_thrE, lccS, iccS, eta, nc, nₚ, nd, Nd, ns, tin, t)
        
        
        m[lnm,:] = m_loc
    #=--------------------------------------EXTRACTION------------------------------------------------=#
    elseif  type_of_move == "extraction"
        ceot, col, eta = extraction(over_thrE, eta)
        
        for i = 1:nodes
            if i ∈ lcc[ceot]
                n[i] += 1
            end
            n[i]
        end
        n
        
        eta_ = [eta[nn_[i]] for i = 1:length(nn_)]
        
        rho_tot -= length(lcc[ceot])/nodes    #change the density after an extraction
        #rho_i[col] -= length(lcc[ceot])/nodes
        
        # array of sites to update
        lnm = unique(nn[lcc[ceot], :]); lnm = [lnm[i] for i = 1:length(lnm) if lnm[i]!=0] 
        #println("lnm ext")
        m_loc = create_m_loc(lnm, eta) #the same for insertion, diffusion and extraction
        #println("m_loc ext \n", m_loc,"\n")
        sd, isd =update_local_sd_isd(lnm, m, m_loc, sd, isd) # the same for insertion, diffusion and extraction
        el, pel = update_local_el_pel_extraction(ceot, el, pel, lcc)
        lcc, cc, icc, over_thrE, lccS, iccS, nₚ, nd, Nd, ns, Next, next, tin, τₚ= update_local_lcc_cc_icc_extraction(
                                       ceot, lcc, cc, icc, over_thrE, lccS, iccS, nₚ, nd, Nd, ns, Next, next, tin, t, τₚ)
        m[lnm,:] = m_loc
    end
    
    densit = push!(densit, rho_tot)
    
    #println("state of the system (eta)")
    #println(eta, "\n")
    
    #println(" \n Total density: ", rho_tot)
    #println("Density for each species \n", rho_i, "\n")   
    #println("list of elements to change list of elements to be modified (lnm) \n:", lnm) 
    #println("particles in the configurations ", mp ,"(sd):\n", sd, "\n")
    #println("positions of moving particles inside sd, -1 if no move is possible (isd)\n", isd, "\n")
    #println("empty sites (el): \n", el, "\n")
    #println("positions of the empty sites in el (pel): \n", pel, "\n")
    #println("clusters (lcc): \n", lcc )
    #println("time formation cluster: ", tin)
    #println("belonging cluster for each particle (cc): \n",cc, "\n")
    #println("positon of a single particle inside the cluster (icc): \n",icc, "\n")
    #println("perimeters of clusters (lccS): \n", lccS, "\n" )
    #println("positon of a single particle inside the cluster perimeter (iccS): \n",iccS, "\n")
    #println("clusters bigger or equal the size: ", thrE, "(over_thrE)\n", over_thrE, "\n")
    #println("np(*nodes): ", nₚ*nodes, "   nd(*nodes): ", nd*nodes, "  Nd: ", Nd, "  ns(*nodes): ", ns*nodes,"   Next: ", Next, "  next(*nodes): ", next*nodes)
    #println("Nin: ", Nin)
    #println("Next: ", Next, "    τₚ: ", τₚ)
    #println("Ndis: ", Ndis, "    τᵤ: ", τᵤ, "\n")
    #println("cont:  ", cont)      
    if ncol == 1
        cont, sum_dt, sum_rho_dt, sum_rho2_dt, sum_rho3_dt, 
        sum_rho4_dt, sum_nₚ_dt, sum_nd_dt, sum_Nd_dt, sum_ns_dt,
        Next, next, sum_τₚ, τₚ, 
        Ndis, sum_τᵤ, τᵤ, Nin = statistics_1species(eta, t, rho_tot, nₚ, nd, Nd, ns, Next, next, cont, 
                                        dt, sum_dt, sum_rho_dt, sum_rho2_dt, sum_rho3_dt, sum_rho4_dt, 
                                        sum_nₚ_dt, sum_nd_dt, sum_Nd_dt, sum_ns_dt, 
                                        sum_τₚ, τₚ, Ndis, sum_τᵤ, τᵤ, Nin)
    
    #else
    #    cont, sum_dt, sum_rho_dt, sum_rho2_dt,sum_rho3_dt, 
    #    sum_rho4_dt, sum_rho_i1_dt, sum_rho_i2_dt, sum_rho_i12_dt, 
    #    sum_nₚ_dt, sum_nd_dt, sum_Nd_dt, sum_ns_dt, 
    #    Next, next, sum_τₚ, τₚ, 
    #    Ndis, sum_τᵤ, τᵤ, Nin = statistics_Nspecies(eta, t, rho_tot, nₚ, nd, Nd,ns, Next, next, rho_i, cont, 
    #                       dt, sum_dt, sum_rho_dt, sum_rho2_dt, sum_rho3_dt, sum_rho4_dt,sum_rho_i1_dt, 
    #                       sum_rho_i2_dt, sum_rho_i12_dt, sum_nₚ_dt, sum_nd_dt, sum_Nd_dt, sum_ns_dt,
    #                       sum_τₚ, τₚ, Ndis, sum_τᵤ, τᵤ, Nin)
    end  
    
    data = "Time: $t \n" * "State: $eta \n" * "Densità: $rho_tot \n" * "Cluster: $lcc \n" * "Numero inserimenti: $Nin \n" * "Numero estrazioni: $Next \n" * "Numero di estrazioni sui nodi: $n \n" * "\n" 
    file = open("cluster_with_$(nodes)_nodes.txt", "w")
    write(file, data)
    #flush(file)
    close(file)

    τₚ = 0.
    τᵤ = 0.
    #println("sum_τₚ: ", sum_τₚ)
    dt = -log(rand(Float64))/sum(q)
    t = t + dt
end
#close(file)
n

In [ ]:
file = open("cluster_with_8810_nodes.txt", "r")
read(file)
close(file)

In [ ]:
t_ = collect(LinRange(0,T,length(densit)));

In [ ]:
plot(t_, densit, "b-")
ylabel("Density")
xlabel("Time")